In [41]:
city = cities[index_of('Bozeman')]
city

{'name': 'Bozeman', 'zoning_field': 'ZONING', 'county': 'Gallatin', 'residential_zones': ['R-S', 'R-1', 'R-2', 'R-3', 'R-4', 'R-5', 'R-O', 'RMH'], 'gdb': 'cities\\Bozeman.gdb', 'layer': 'cities\\Bozeman.gdb\\Parcels_Bozeman'}

In [44]:
codeblock = """
def eval_zoning(zone):
    if zone in ['R-S', 'R-1', 'RMH']:
        return 1
    elif zone == 'R-2':
        return 2
    elif zone == 'R-3':
        return 3
    elif zone in ['R-4', 'R-5', 'R-O']:
        return 5
    else:
        return 0
"""

arcpy.management.CalculateField(
    city['layer'], 
    "zoning_allows", 
    "eval_zoning(!zoning!)", 
    "PYTHON3", 
    codeblock, 
    "SHORT"
)

<Result 'cities\\Bozeman.gdb\\Parcels_Bozeman'>

In [45]:
codeblock = """
def eval_lot_size(zone, sf):
    if zone in ['R-S', 'R-1']:
        if sf >= 4000:
            return 1
        else:
            return 0
    elif zone in ['R-2']:
        if sf >= 5000:
            return 2
        elif sf >= 4000:
            return 1
        else:
            return 0
    elif zone in ['R-3']:
        if sf >= 9000:
            return 3
        if sf >= 5000:
            return 2
        if sf >= 4000:
            return 1
        else:
            return 0
    elif zone in ['R-4', 'R-O']:
        if sf >= 12000:
            return 4
        if sf >= 9000:
            return 3
        if sf >= 5000:
            return 2
        if sf >= 4000:
            return 1
        else:
            return 0
    elif zone in ['R-5']:
        if sf >= 5000:
            return 5
        if sf >= 3000:
            return 1
        else:
            return 0
    else:
        return 0
"""

arcpy.management.CalculateField(
    city['layer'], 
    "lot_size_allows", 
    "eval_lot_size(!zoning!, !lot_size!)", 
    "PYTHON3", 
    codeblock, 
    "SHORT"
)

<Result 'cities\\Bozeman.gdb\\Parcels_Bozeman'>

In [46]:
codeblock = """
def eval_lot_width(zone, w):
    if zone in ['R-S', 'R-1']:
        if w >= 50:
            return 1
        else:
            return 0
    elif zone in ['R-2']:
        if w >= 60:
            return 2
        elif w >= 50:
            return 1
        else:
            return 0
    elif zone in ['R-3']:
        if w >= 60:
            return 3
        if w >= 50:
            return 1
        else:
            return 0
    elif zone in ['R-4', 'R-O']:
        if w >= 60:
            return 4
        if w >= 50:
            return 2
        else:
            return 0
    elif zone in ['R-5']:
        if w >= 50:
            return 5
        if w >= 35:
            return 1
        else:
            return 0
    else:
        return 0
"""

arcpy.management.CalculateField(
    city['layer'], 
    "lot_width_allows", 
    "eval_lot_width(!zoning!, !lot_width!)", 
    "PYTHON3", 
    codeblock, 
    "SHORT"
)

<Result 'cities\\Bozeman.gdb\\Parcels_Bozeman'>

In [47]:
codeblock = """
def categorize(zone):
    if zone in ['R-2', 'R-3', 'R-4', 'R-5', 'R-O']:
        return "Penalized Multifamily"
    elif zone in ['R-1', 'R-S', 'RMH']:
        return "Single-Family"
"""
arcpy.management.CalculateField(
    city['layer'], 
    "category", 
    "categorize(!zoning!)",
    "PYTHON3",
    codeblock,
    "TEXT"
)

<Result 'cities\\Bozeman.gdb\\Parcels_Bozeman'>